In [ ]:
# =========================
# IMPORTS & SETUP
# =========================

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, confusion_matrix
import seaborn as sns
import time, os, sys, random

# Add parent utils path if needed
sys.path.append('../Utils')  
import utils as ut
import configs

# Reproducibility
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f" Using device: {device}")


Using device: cpu


In [ ]:
# =========================
# DATA LOADING & TRANSFORMS (WITH AUGMENTATION)
# =========================

#  Training transforms (augmentation)
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),              # First resize to a bit larger
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Random crop for variability
    transforms.RandomHorizontalFlip(p=0.5),     # Flip left-right
    transforms.RandomVerticalFlip(p=0.2),       # Sometimes flip upside-down
    transforms.RandomRotation(15),              # Slight random rotation
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02),  # Color augmentation
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)), # Small translations
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

#  Validation & test transforms (no augmentation, only normalization)
eval_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load dataset with proper transforms
full_dataset = datasets.ImageFolder(configs.MULTIVIEW_TRAIN_DIR, transform=train_transform)
test_dataset = datasets.ImageFolder(configs.MULTIVIEW_TEST_DIR, transform=eval_transform)

# Train/val split
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Override val transform (no augmentation)
val_dataset.dataset.transform = eval_transform  

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

print(f" Train size: {len(train_dataset)}, Val size: {len(val_dataset)}, Test size: {len(test_dataset)}")
NUM_CLASSES = len(full_dataset.classes)
print(f" Number of classes: {NUM_CLASSES}")


In [ ]:
# =========================
# 1. MODEL - FULL FINE-TUNING
# =========================
class ResNetFullFineTuner(nn.Module):
    def __init__(self, num_classes):
        super(ResNetFullFineTuner, self).__init__()
        self.model = models.resnet18(pretrained=True)

        #  Unfreeze all layers for full fine-tuning
        for param in self.model.parameters():
            param.requires_grad = True

        # Replace final FC layer for our dataset
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.model(x)



In [ ]:
# =========================
# 2. TRAINING FUNCTION
# =========================
def train_model(model, train_loader, val_loader, epochs=25, lr=1e-4):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

    history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

    for epoch in range(epochs):
        model.train()
        running_loss, correct, total = 0, 0, 0

        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / len(train_loader)
        train_acc = correct / total
        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)

        #  Validation phase
        model.eval()
        val_running_loss, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()
                preds = outputs.argmax(dim=1)
                val_correct += preds.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss = val_running_loss / len(val_loader)
        val_acc = val_correct / val_total
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(f" Epoch [{epoch+1}/{epochs}] | "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        scheduler.step()

    return model, history



# =========================
# 3. EVALUATION FUNCTION
# =========================
def evaluate_model(model, data_loader, device, return_only_acc=False):
    model.eval()
    all_preds, all_labels = [], []
    correct, total = 0, 0

    with torch.no_grad():
        for imgs, labels in data_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)

    acc = correct / total
    if return_only_acc:
        return acc
    return acc, all_preds, all_labels


# =========================
# 4. PLOTTING FUNCTION
# =========================
def plot_history(history):
    plt.figure(figsize=(12,5))

    plt.subplot(1,2,1)
    plt.plot(history["train_loss"], label="Train Loss")
    plt.plot(history["val_loss"], label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(history["train_acc"], label="Train Acc")
    plt.plot(history["val_acc"], label="Val Acc")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()

    plt.show()



In [ ]:
# =========================
# RUN TRAINING
# =========================
model = ResNetFullFineTuner(num_classes=NUM_CLASSES).to(device)
model, history = train_model(model, train_loader, val_loader, epochs=25, lr=1e-4)

# =========================
# EVALUATE ON TEST SET
# =========================
test_acc, all_preds, all_labels = evaluate_model(model, test_loader)
print(f"\n Final Test Accuracy: {test_acc:.4f}")
print("Classification Report:\n", classification_report(all_labels, all_preds, target_names=full_dataset.classes))

# =========================
# CONFUSION MATRIX
# =========================
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=full_dataset.classes,
            yticklabels=full_dataset.classes)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()

# =========================
# PLOT TRAINING CURVES
# =========================
plot_history(history)